#  Mission Statement
​
Analyze smart device usage data in order to gain insight into how consumers use non-Bellabeat smart devices.
Try and apply the learnings to one of Bellabeat’s products.
​
* What are some trends in smart device usage?
* How could these trends apply to Bellabeat customers?
* How could these trends help influence Bellabeat marketing strategy?

# Ask

Considering who my Stakeholders, and Key-Stakeholders especially, are:

Key Stakeholders:

* Founders/Owners
* Urška Sršen: Bellabeat’s cofounder and Chief Creative Officer
* Sando Mur: Mathematician and Bellabeat cofounder; key member of the Bellabeat executive team

Other Stakeholders include:

* Team: Bellabeat marketing analytics A team of data analysts responsible for collecting, analyzing, and reporting data that helps guide Bellabeat’s marketing strategy. In this scenario I joined this team six months ago and have been busy learning about Bellabeat’’s mission and business goals — as well as how yI, as a junior data analyst, can help Bellabeat achieve them.
* ! added myself: The Marketing Dept. - identifying their users, knowing their behaviors developing matching product propositions and advertising them to users and potential users
* ! added myself: App- and hardware developers - building the products we recommend hands on

# The business task 
(as I see it)
IDEALLY users consult Bellabeat’s products actively and acceptingly. They are scheduling daily, weekly and cyclically for activities, relaxation and health related decision making. Users feel health benefits and report good overall well being. They confidently use the help of Bellabeat products and also recommend doing so to others. Bellabeat is a positive part of the users’ lives. So:
Let’s find what’s missing yet and identify measures to bridge this gap!

So what I’ll try and do is: **find patterns of engagement**
* Who’s sharing,
* What data were users giving up?
* What did they not disclose?


also consider:
* how use-able is the info (quality/data)?
* What do other sources suggest?

Measures I’m looking for should address **how Bellabeat might intensify engagement**.

# Prepare
Download the data from it’s source origin: https://zenodo.org/record/53894/#.Y4TZ0HbMJPY
please acknowledge / cite Furberg, Robert; Brinton, Julia; Keating, Michael ; Ortiz, Alexa (RTI International) the data contains: “ datasets (...) generated by respondents to a distributed survey via Amazon Mechanical Turk between 03.12.2016-05.12.2016.  Thirty eligible Fitbit users consented to the submission of personal tracker data, including minute-level output for physical activity, heart rate, and sleep monitoring.

I went on to:
* store the originals, create working copies to edit
* comb through the data to learn how it’s structured and which items need check-up/clean up
* 
Data: collection of 18 CSV files ( 'dailyActivity_merged.csv', 'dailyCalories_merged.csv, ''dailyIntensities_merged.csv', 'dailySteps_merged.csv', 'heartrate_seconds_merged.csv', 'hourlyCalories_merged.csv', 'hourlyIntensities_merged.csv', 'hourlySteps_merged.csv', 'minuteCaloriesNarrow_merged.csv','minuteCaloriesWide_merged.csv', 'minuteIntensitiesNarrow_merged.csv', 'minuteIntensitiesWide_merged.csv', 'minuteMETsNarrow_merged.csv, ''minuteSleep_merged.csv', 'minuteStepsNarrow_merged.csv', 'minuteStepsWide_merged.csv', 'sleepDay_merged.csv, ''weightLogInfo_merged.csv' )

A substantial part of the data contains detailed minute by minute breakdowns of info-segments. For our specific analysis these seem overly granular and not to contain much useful insight so far. Many items are also included and come pre-merged into the dailyActivity_merged sheet.
Unique and self contained useful info seem to be mostly in the following sheets I will therefore be using:

* dailyActivity_merged
* sleepDay_merged
* weightLogInfo_merged

The data’s first integrity check up: 
* it’s Fitbit data which is hardware not made by Bellabeat therefore application is limited
* there are 33 respondents and 30 days recorded, this seems a small sample
* the data was gathered in 2016 which makes it old and potentially irrelevant
* much of the tracked items might have suffered from incomplete or technically insufficiently reliable methods like calories; others like sleep at least have to be considered potentially flawed; reports from the time may suggest that distance traveled by means other than on foot may have been recorded as well in “steps”


# Process
What tools are you choosing and why?
* Spreadsheets in order to see entries in their respective cells to weigh whether the data is usable;
* SQL (Big Query) to analyze per query
​
data integrity
* date stamps/formats have been resolved
* varying entry formats have been unified
* rows w/ date entries outside the study’s time frame were removed
* All original data were preserved in original tables
* plausible fixes f. steps and distance as documented
​

# Analyze
Checking for the following insights by query:


* Abstract 1: Gross Averages
All users: how many steps did they take per day, how many minutes did they commit dta (per day), on how many dates did they commit data: 

#@bigquery

SELECT
  ROUND (AVG (steps), 2) AS Daily_StepsAverage,
  ROUND (AVG (minutes), 2) AS Daily_MinutesAverage,
  ROUND (AVG (dates), 2) AS Commit_DatesAverage
From(
  SELECT
    steps,
    minutes,
    dates
  
  FROM(
    SELECT
    Id,
    AVG (edit_TotalSteps) AS steps,
    AVG (edit_TotalMinutes) AS minutes,
    count(DISTINCT ActivityDate) AS dates
      
FROM `fleet-day-378718.BellaBeat_DailyActivities.dailyActivities`
          group by Id
  )
)

* In order to see individual contributions (days) and the users' avg daily steps taken:

#@bigquery
SELECT

  Id,
  
  ROUND(stepsAvg, 2) AS steps,
  
  dates,
  
  stepsTotal,

FROM (


  SELECT
    
    Id,
    
    sum (edit_TotalMinutes) AS sum_totalMinutes,
    
    count(DISTINCT ActivityDate) AS dates,
    
    sum (edit_TotalSteps) AS stepsTotal,
    
    AVG (edit_TotalSteps) AS stepsAvg
    
    
  FROM `fleet-day-378718.BellaBeat_DailyActivities.dailyActivities`
  
  group by Id
)

order by sum_totalMinutes DESC, stepsAvg DESC

The most active quarter of "steppers" (users w/ most average steps per day) and their contributed days

#@bigquery
SELECT
  ROUND(AVG(dates), 2) AS datesContributedTop8,
  ROUND(AVG(steps), 2) AS stepsTop8

FROM(

SELECT
  Id,
  ROUND(stepsAvg, 2) AS steps,
  dates,

FROM(
  SELECT
    Id,
    sum (edit_TotalMinutes) AS sum_totalMinutes,
    sum (edit_TotalSteps) AS stepsTotal,
    AVG (edit_TotalSteps) AS stepsAvg,
    count(DISTINCT ActivityDate) AS dates

  FROM `fleet-day-378718.BellaBeat_DailyActivities.dailyActivities`
  group by Id
)
order by steps DESC
LIMIT 8
)

so on average users commited on 25 days total
compared to
28.4 days during the trial period of the most active "steppers"
(averaging 12548 daily steps over 8363 steps/gross average)

When quering for the segment of the most actively commiting (days)

#@bigquery
SELECT
  ROUND(AVG(dates), 2) AS datesContributed,
  ROUND(AVG(steps), 2) AS stepsMostContributedDates

FROM(

SELECT
  Id,
  ROUND(stepsAvg, 2) AS steps,
  dates,

FROM(
  SELECT
    Id,
    sum (edit_TotalMinutes) AS sum_totalMinutes,
    sum (edit_TotalSteps) AS stepsTotal,
    AVG (edit_TotalSteps) AS stepsAvg,
    count(DISTINCT ActivityDate) AS dates

  FROM `fleet-day-378718.BellaBeat_DailyActivities.dailyActivities` 
  group by Id
)
order by dates DESC
LIMIT 8
)

The two showing a somewhat strong correlation between most segments of engagement
steps recorded and days actively commited

While quering for the least active "steppers" and the no. days they commited

#@bigquery
SELECT
  ROUND(AVG(dates), 2) AS datesContributedBottom8,
  ROUND(AVG(steps), 2) AS stepsBottom8

FROM(

SELECT
  Id,
  ROUND(stepsAvg, 2) AS steps,
  dates,

FROM(
  SELECT
    Id,
    sum (edit_TotalMinutes) AS sum_totalMinutes,
    sum (edit_TotalSteps) AS stepsTotal,
    AVG (edit_TotalSteps) AS stepsAvg,
    count(DISTINCT ActivityDate) AS dates

  FROM `fleet-day-378718.BellaBeat_DailyActivities.dailyActivities`
  group by Id
)
order by steps ASC
LIMIT 8
)

While showing roughly the same spread w/ the no. steps taken in comparison to avrage number of steps into the other direction,
the number of commited days ais also significantly lower by a 4-day margin compared to the gross avearge.

When inspecting the users who contribute the least no. days

#@bigquery
SELECT
  ROUND(AVG(dates), 2) AS datesContributedLeast,
  ROUND(AVG(steps), 2) AS stepsBottomFromFewestDates

FROM(

SELECT
  Id,
  ROUND(stepsAvg, 2) AS steps,
  dates,

FROM(
  SELECT
    Id,
    sum (edit_TotalMinutes) AS sum_totalMinutes,
    sum (edit_TotalSteps) AS stepsTotal,
    AVG (edit_TotalSteps) AS stepsAvg,
    count(DISTINCT ActivityDate) AS dates

  FROM `fleet-day-378718.BellaBeat_DailyActivities.dailyActivities`
  group by Id
)
order by dates ASC
LIMIT 8
)

Showing a substantially lower number of days (another roughly 4 days) commited while stepping more than 20% more daily averages than the least active "steppers".

#  Intermediate conclusion:
​
Engagement with the device and physical activity showed correlations as follows:
​
* The top quarter of the physically most active users was very closely related/strongly overlapping w/ the segment of the users commiting on most of the days of the trial.
* Engagement from users who were physically less active was lower by a substantial margin.
* However, the least engaged users were on average not the least active ones.

#  Quantifying engagement
​
Engagement with the device by all committed days per user including their sleep days and weight inputs:

#@bigquery
SELECT
  User_IDs,
  dates AS steps_Dates,
  SleepCommit AS sleep_Dates,
  weightCommits AS weight_Dates,
  (dates + SleepCommit + weightCommits) AS TotalDates_Commits,
  (CAST (SleepMinutes AS int) + Daily_ActiveMinutes_Avg) AS TotalMinutes_Commits  

FROM(

SELECT
  User_IDs,
  dates,
  SleepCommit,
  weightCommits,
  CAST (sum_totalMinutes AS int ) AS Daily_ActiveMinutes_Avg,
  IFNULL(MinsAsleep , 0) AS SleepMinutes

FROM(
  SELECT
    `fleet-day-378718.BellaBeat_DailyActivities.dailyActivities`.Id AS User_IDs,
    AVG (`fleet-day-378718.BellaBeat_DailyActivities.dailyActivities`.edit_TotalMinutes) AS sum_totalMinutes,
    count(DISTINCT `fleet-day-378718.BellaBeat_DailyActivities.dailyActivities`.ActivityDate) AS dates,
    Count (DISTINCT `fleet-day-378718.BellaBeat_DailyActivities.new_sleepDay`.SleepDay) AS SleepCommit,
    AVG (`fleet-day-378718.BellaBeat_DailyActivities.new_sleepDay`.TotalMinutesAsleep) AS MinsAsleep,
    Count (DISTINCT `fleet-day-378718.BellaBeat_DailyActivities.weightInfo`.Date) AS weightCommits
  FROM `fleet-day-378718.BellaBeat_DailyActivities.dailyActivities` 

LEFT JOIN `fleet-day-378718.BellaBeat_DailyActivities.new_sleepDay`
ON `fleet-day-378718.BellaBeat_DailyActivities.new_sleepDay`.Id = `fleet-day-378718.BellaBeat_DailyActivities.dailyActivities`.Id
LEFT JOIN `fleet-day-378718.BellaBeat_DailyActivities.weightInfo`
ON `fleet-day-378718.BellaBeat_DailyActivities.weightInfo`.Id  = `fleet-day-378718.BellaBeat_DailyActivities.dailyActivities`.Id
GROUP BY `fleet-day-378718.BellaBeat_DailyActivities.dailyActivities`.Id
)
GROUP BY User_IDs, SleepCommit, dates, weightCommits, MinsAsleep,  sum_totalMinutes
)
order by TotalDates_Commits DESC

13 of 33 users commited a total number of 50 or more entries to their sleep,- weight- and/or daily activities,
9 users did commit less than 30 entries total
The number of dates was usually dominated by the activity dates,
followed by sleep dates and very few daily entries for weight. (the entry numbers of 28 and 22 daily entries for two resprective users are great outliers here).

#  Intermediate conclusion (2) :
​
Committing entries to the device 
​
* More than a third of the users very actively used the device more or less daily for the automated entries
* Weight entries occured only sparsely except for two individual cases.
* The most actively commiting users for sleep days were also showing high engagement commiting high no. dates fo rdaily activities

#  Conclusions:
​
Some trends to be identified with the data that's available (see Prepare section for information on limitations)
​
* Users who are very physically active also engaged a lot more with the device during the trial period generally. 
* Automated recordings/inputs were grossly more represented than manual inputs.
* Less physical activity correlated with fewer entries/commits total - yet the least active physically were not the least engaged of users. Least interested in engaging were a group of low/average activity.

#  Actions:
​
When assuming some applicability of the data what actions could be useful in order to achieve better results for users as outlined in the business case? -  all recommendations address the Leaf device in tight conjunction with the BellaBeat-App
​
* The best interaction user < > device seems to happen when interaction is frictionless and automated. Therefore device features should focus on easy, intuitive inputs generated to the device, responding to activity, sleep etc. and possibly discard other functions.

* Steps/physical action apparently define the users' willingness to engage. Keeping up interest in their activity readings seems crucial to keep engagement high. Possibly a gamified interaction - taking the stepper/commiter-'personality' into account when addressing the user - could be helpful via a mobile Id/account.

* The physically least active users are not identical with the laziest users as engagement is concerned. Therefore addressing users currently more challenged to take more steps or be more active seems possible. My suggestion would be to get an incremental improvement feature, allowing for gradual, small-step-improvements to be communicated. In addition, well proven concepts for time management could be communicated via App and incremental progress should be rewarded either a) automated app feature or b) (preferred) community feature of the app.